TERAPIE INTENSIVE IN ITALIA

In [5]:
!wget https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv 

--2021-02-08 08:50:41--  https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40957 (40K) [text/plain]
Saving to: ‘dpc-covid19-ita-andamento-nazionale.csv.1’

dpc-covid19-ita-and 100%[===================>]  40.00K  --.-KB/s    in 0.005s  

2021-02-08 08:50:41 (7.67 MB/s) - ‘dpc-covid19-ita-andamento-nazionale.csv.1’ saved [40957/40957]



In [6]:
!pip3 install plotly

In [7]:
import os
import plotly.graph_objects as go

GRAFICO TERAPIA INTENSIVA

In [8]:
#ITALIA

covid19_italia = open ("dpc-covid19-ita-andamento-nazionale.csv", "r").readlines()

covid19_italia = [i.split(",") for i in covid19_italia]

data = [i[0] for i in covid19_italia]
terapia_intensiva = [i[3] for i in covid19_italia]

print ("TOTALE TERAPIE INTENSIVE:", terapia_intensiva[-1])

print ("TOTALE TERAPIE INTENSIVE OGGI: ", data[-1][:10], ": ", int(terapia_intensiva[-1]) - int(terapia_intensiva[-2]))

fig = go.Figure(data=[
    go.Bar(name='data', x=data, y=terapia_intensiva)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

TOTALE TERAPIE INTENSIVE: 2107
TOTALE TERAPIE INTENSIVE OGGI:  2021-02-07 :  -3


GRAFICO TERAPIA INTENSIVA ULTIMI 60 GIORNI

In [15]:
fig = go.Figure(data=[
    go.Bar(name='data', x=data, y=terapia_intensiva[-60:])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [ ]:
!pip3 install pystan
!pip3 install fbprophet

In [ ]:
import pandas as pd
import fbprophet
from fbprophet import Prophet

In [16]:
starting_data = "2020-10-01"
predicted_days = 7

for i in data:
    if starting_data in i:
        observation_number = data.index(i)
        observation_number = len(data) - observation_number
        
variable_0 = "data"
variable_1 = data[-(observation_number):]
variable_2 = "totale_terapie_intensive_odierne"
variable_3 = terapia_intensiva[-(observation_number):]

df = pd.DataFrame(
    {variable_0: variable_1,
     variable_2: variable_3
    })

df.columns = ["ds","y"]

model = Prophet(yearly_seasonality=False,weekly_seasonality=False, daily_seasonality=False)
model.fit(df)

future = model.make_future_dataframe(periods=predicted_days) #, freq='D'

pred = model.predict(future)

pred[-(predicted_days):]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
130,2021-02-08 17:00:00,2085.596237,2058.099414,2113.513951,2085.596237,2085.596237,0.0,0.0,0.0,0.0,0.0,0.0,2085.596237
131,2021-02-09 17:00:00,2066.220660,2035.675693,2099.914487,2062.789290,2072.506820,0.0,0.0,0.0,0.0,0.0,0.0,2066.220660
132,2021-02-10 17:00:00,2046.845083,2012.558876,2083.777366,2032.730699,2063.708410,0.0,0.0,0.0,0.0,0.0,0.0,2046.845083
133,2021-02-11 17:00:00,2027.469506,1986.413499,2071.244660,2001.012105,2057.158945,0.0,0.0,0.0,0.0,0.0,0.0,2027.469506
134,2021-02-12 17:00:00,2008.093929,1959.635769,2061.650503,1967.131805,2056.619370,0.0,0.0,0.0,0.0,0.0,0.0,2008.093929
135,2021-02-13 17:00:00,1988.718352,1925.796856,2058.447005,1932.530232,2055.517366,0.0,0.0,0.0,0.0,0.0,0.0,1988.718352
136,2021-02-14 17:00:00,1969.342775,1889.049223,2052.308765,1893.787353,2052.908412,0.0,0.0,0.0,0.0,0.0,0.0,1969.342775


In [17]:
date_pred = list(pred['ds'])
date_pred = (date_pred[-(predicted_days):])

yhat = list(pred['yhat'])
yhat = (yhat[-(predicted_days):])
yhat = [round(i,2) for i in yhat]

yhat_lower = list(pred['yhat_lower'])
yhat_lower = (yhat_lower[-(predicted_days):])
yhat_lower = [round(i,2) for i in yhat_lower]

yhat_upper = list(pred['yhat_upper'])
yhat_upper = (yhat_upper[-(predicted_days):])
yhat_upper = [round(i,2) for i in yhat_upper]

date_yhat = zip(date_pred, yhat_lower, yhat, yhat_upper)
date_yhat = (str(list(date_yhat))).replace (",","\n")
print (date_yhat)

fig = go.Figure(data=[
    go.Bar(name='yhat_lower', x=date_pred, y=yhat_lower),
    go.Bar(name='yhat', x=date_pred, y=yhat),
    go.Bar(name='yhat_upper', x=date_pred, y=yhat_upper)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

[(Timestamp('2021-02-08 17:00:00')
 2058.1
 2085.6
 2113.51)
 (Timestamp('2021-02-09 17:00:00')
 2035.68
 2066.22
 2099.91)
 (Timestamp('2021-02-10 17:00:00')
 2012.56
 2046.85
 2083.78)
 (Timestamp('2021-02-11 17:00:00')
 1986.41
 2027.47
 2071.24)
 (Timestamp('2021-02-12 17:00:00')
 1959.64
 2008.09
 2061.65)
 (Timestamp('2021-02-13 17:00:00')
 1925.8
 1988.72
 2058.45)
 (Timestamp('2021-02-14 17:00:00')
 1889.05
 1969.34
 2052.31)]
